In [31]:
import setup # resolve path to 'src'
import numpy as np
import pandas as pd

from typing import Optional
from build_parse import *
from metrics import *

pd.set_option('display.max_rows', None)

In [32]:
progs = [ CoreutilsProgram(progname) for progname in COREUTILS_PROG_NAMES ]
prognames = [ prog.get_name() for prog in progs ]

prognames_analyze = [ "stat", "nohup", "pinky", "csplit", "ginstall", "fmt", "df", "join", "expr", "seq", "unexpand", "tsort", "tee", "base64", "sum", "cksum", "wc" ]
progs_analyze = []
for progname in prognames_analyze:
    for prog in progs:
        if progname == prog.get_name():
            progs_analyze.append(prog)
            break

def prog_from_progname(progname: str) -> Program:
    for prog in progs:
        if progname == prog.get_name():
            return prog

# Define the build options to test for each program
debug_opts = BuildOptions(debug=True, strip=False, optimization=0)
standard_opts = BuildOptions(debug=False, strip=False, optimization=0)
strip_opts = BuildOptions(debug=False, strip=True, optimization=0)

opts_sets = (strip_opts, standard_opts, debug_opts)
opts_sets_keys = ("strip", "standard", "debug")

# Get the parser functions
dwarf_parser = get_parser("dwarf")
ghidra_parser = get_parser("ghidra")

varnode_compare_levels = list(VarnodeCompareLevel.range())
varnode_compare_level_labels = [ "Varnodes matched @ level={}".format(VarnodeCompareLevel.to_string(level)) for level in VarnodeCompareLevel.range() ]

primitive_metatypes = [MetaType.INT, MetaType.FLOAT, MetaType.POINTER]
complex_metatypes = [MetaType.ARRAY, MetaType.STRUCT, MetaType.UNION]
metatypes = primitive_metatypes + complex_metatypes
metatype_labels = [ MetaType.repr(metatype) for metatype in metatypes ]

# ensure that each program is built according to all variations of build options
for prog in progs:
    for opts in (debug_opts, standard_opts, strip_opts):
        assert(prog.valid_build(opts))

In [33]:

# the filename format for saving parsed ProgramInfo pickle objects
def mangle_proginfo_save_name(parsername: str, prog: Program, opts: BuildOptions) -> str:
    return "{}.{}.pickle".format(prog.get_binary_name(opts), parsername)

def get_proginfo_save_path(parsername: str, prog: Program, opts: BuildOptions) -> Path:
    return PICKLE_CACHE_DIR.joinpath(mangle_proginfo_save_name(parsername, prog, opts))

def save_proginfo(proginfo: ProgramInfo, parsername: str, prog: Program, opts: BuildOptions):
    save_pickle(proginfo, get_proginfo_save_path(parsername, prog, opts))

def load_proginfo(parsername: str, prog: Program, opts: BuildOptions) -> ProgramInfo:
    return load_pickle(get_proginfo_save_path(parsername, prog, opts))

# the filename format for saving UnoptimizedProgramInfoCompare2 objects
def mangle_cmp_save_name(prog: Program, opts: BuildOptions) -> str:
    return "{}.cmp.pickle".format(prog.get_binary_name(opts))

def get_cmp_save_path(prog: Program, opts: BuildOptions) -> Path:
    return PICKLE_CACHE_DIR.joinpath(mangle_cmp_save_name(prog, opts))

def save_cmp(cmp: UnoptimizedProgramInfoCompare2, prog: Program, opts: BuildOptions):
    save_pickle(cmp, get_cmp_save_path(prog, opts))

def load_cmp(prog: Program, opts: BuildOptions) -> UnoptimizedProgramInfoCompare2:
    return load_pickle(get_cmp_save_path(prog, opts))

In [34]:
union_prognames = ("cp", "date", "factor", "mv", "od", "ptx")
union_progs = [ CoreutilsProgram(progname) for progname in union_prognames ]

In [35]:
# DWARF: only parse with the debug build options
# Ghidra: parse with all variations of build options
# Cache the results in local pickle_cache directory, named based on the 'mangle' scheme

reparse = False # should we re-parse even if we already parsed and cached a program?
skip_parsing = True # should we skip the parsing? set to True if we already parsed & cached

class ParseException(Exception):
    pass

def parse(parser: Callable, prog: Program, opts: BuildOptions) -> Optional[ProgramInfo]:
    try:
        return parser(prog.get_binary_path(opts))
    except:
        return None

failed = []
if not skip_parsing:
    for prog in union_progs:
        dwarf_debug_savepath = get_proginfo_save_path("dwarf", prog, debug_opts)
        if reparse or not dwarf_debug_savepath.exists():
            dwarf_debug = parse(dwarf_parser, prog, debug_opts)
            if dwarf_debug is None:
                failed.append(("dwarf", prog.get_name(), debug_opts))
            else:
                save_pickle(dwarf_debug, dwarf_debug_savepath)

        for opts in opts_sets:
            ghidra_parse_savepath = get_proginfo_save_path("ghidra", prog, opts)
            if reparse or not dwarf_debug_savepath.exists():
                ghidra_parse = parse(ghidra_parser, prog, opts)
                if ghidra_parse is None:
                    failed.append(("ghidra", prog.get_name(), opts))
                else:
                    save_pickle(ghidra_parse, ghidra_parse_savepath)

In [36]:
print(failed)

for prog in progs:
    for opts in opts_sets:
        assert(get_proginfo_save_path("ghidra", prog, opts).exists())

[]


In [37]:
# For each program & build options combination, compute & store comparison object

recompare = False
skip_comparisons = True

failed = []
if not skip_comparisons:
    for prog in progs:
        # load the DWARF ground-truth ProgramInfo
        dwarf_proginfo = load_pickle(get_proginfo_save_path("dwarf", prog, debug_opts))
        assert(dwarf_proginfo is not None)

        # for each set of compilation options, load the Ghidra decompiler ProgramInfo
        # then compute & store the comparison object
        for opts in opts_sets:
            cmp_save_path = get_cmp_save_path(prog, opts)
            if recompare or not cmp_save_path.exists():
                ghidra_proginfo = load_pickle(get_proginfo_save_path("ghidra", prog, opts))
                assert(ghidra_proginfo is not None)
                try:
                    cmp = compare2(dwarf_proginfo, ghidra_proginfo)
                    save_pickle(cmp, get_cmp_save_path(prog, opts))
                except:
                    failed.append((prog.get_name(), opts))

In [38]:
print(failed)

for prog in progs:
    for opts in opts_sets:
        assert(get_cmp_save_path(prog, opts).exists())

[]


In [39]:
# For each opts, compute the tables

def mangle_table_save_name(
    tablename: str,
    opts: BuildOptions
) -> str:
    return "{}{}.csv".format(tablename, suffix(opts))

def build_options_display_suffix(opts: BuildOptions) -> str:
    return "(optimization={}, stripped={}, debug={})".format(opts.optimization, opts.strip, opts.debug)

def mangle_table_display_name(
    tablename: str,
    opts: BuildOptions
) -> str:

    return "{} {}".format(tablename, build_options_display_suffix(opts))

def get_table_save_path(
    tablename: str,
    opts: BuildOptions
) -> Path:
    return DATA_DIR.joinpath(mangle_table_save_name(tablename, opts))

def get_table_save_path_generic(
    tablename: str
) -> Path:
    return DATA_DIR.joinpath(tablename + ".csv")

def load_table(
    tablename: str
) -> pd.DataFrame:
    return pd.read_csv(DATA_DIR.joinpath(tablename + ".csv"), index_col=0)

def load_table_opts(
    tablename: str,
    opts: BuildOptions
) -> pd.DataFrame:
    return pd.read_csv(get_table_save_path(tablename, opts), index_col=0)

def load_table_opts_filter_analyzed(tablename: str, opts: BuildOptions) -> pd.DataFrame:
    return load_table_opts(tablename, opts).filter(prognames_analyze, axis=0)

def get_table_from_group(
    grp: MetricsGroup,
    opts: BuildOptions
)-> pd.DataFrame:
    return load_table_opts(grp.get_name(), opts)

In [40]:
metrics_groups = make_metrics()

bytes_group = metrics_groups[0]
functions_group = metrics_groups[1]
varnodes_group = metrics_groups[2]
decomposed_varnodes_group = metrics_groups[9]
array_comparisons_group = metrics_groups[13]

def varnodes_group_metatype(metatype: int) -> MetricsGroup:
    _map = dict([ (meta, i) for i, meta in enumerate(primitive_metatypes + complex_metatypes, 3) ])
    return metrics_groups[_map[metatype]]

varnodes_groups_metatypes = [ varnodes_group_metatype(metatype) for metatype in (primitive_metatypes + complex_metatypes) ]

def decomposed_varnodes_group_metatype(metatype: int) -> MetricsGroup:
    _map = dict([ (meta, i) for i, meta in enumerate(primitive_metatypes, 10) ])
    return metrics_groups[_map[metatype]]

decomposed_varnodes_groups_metatypes = [ decomposed_varnodes_group_metatype(metatype) for metatype in primitive_metatypes ]

high_varnodes_groups = [varnodes_group] + varnodes_groups_metatypes
decomposed_varnodes_groups = [decomposed_varnodes_group] + decomposed_varnodes_groups_metatypes

In [41]:
recompute = False
skip_compute_metrics = True

if not skip_compute_metrics:
    for opts in opts_sets:
        cmps = [ load_cmp(prog, opts) for prog in progs ]
        for grp in metrics_groups:
            save_path = get_table_save_path(grp.get_name(), opts)
            tablename = mangle_table_display_name(grp.get_display_name(), opts)
            print(tablename)
            if recompute or not save_path.exists():
                df = compute_comparisons_metrics_dataframe(prognames, cmps, grp.get_metrics())
                df.to_csv(save_path)


In [42]:
skip_fix_varnode_metrics = False

# Add "Varnodes fraction partially recovered" & "Varnodes fraction exactly recovered" columns
# to the varnodes tables (if not already done)
if not skip_fix_varnode_metrics:
    for grp in high_varnodes_groups + decomposed_varnodes_groups:
        for opts in opts_sets:
            df = get_table_from_group(grp, opts)
            df["Varnodes fraction partially recovered"] = df.iloc[:,2:6].sum(axis=1) / df.iloc[:,0]
            df["Varnodes fraction exactly recovered"] = df.iloc[:,5] / df.iloc[:,0]
            savepath = get_table_save_path(grp.get_name(), opts)
            df.to_csv(savepath)

def get_varnode_group_average_stats(grp: MetricsGroup) -> pd.Series:
    df = get_table_from_group(grp, opts)
    return df.iloc[:,6:].mean(axis=0)

In [43]:
skip_generate_metatype_level_summaries = False

if not skip_generate_metatype_level_summaries:
    for opts in opts_sets:
        raw_seriess = []
        ratios_seriess = []
        for metatype in metatypes:
            metatype_str = MetaType.repr(metatype)
            grp = varnodes_group_metatype(metatype)
            df = get_table_from_group(grp, opts)
            metatype_varnodes = df.iloc[:,0].sum()
            varnodes_by_levels = df.iloc[:,1:6].sum(axis=0)
            varnodes_by_levels.index = varnode_compare_level_labels
            level_ratios = varnodes_by_levels / metatype_varnodes
            raw_seriess.append(varnodes_by_levels)
            ratios_seriess.append(level_ratios)

        high_raw_df = pd.DataFrame(
            raw_seriess,
            index=[ MetaType.repr(metatype) for metatype in metatypes ],
            columns=varnode_compare_level_labels
        )
        high_raw_tablename = "metatype_match_levels"
        high_raw_savepath = get_table_save_path(high_raw_tablename, opts)
        high_raw_df.to_csv(high_raw_savepath)
        
        high_ratios_df = pd.DataFrame(
            ratios_seriess,
            index=[ MetaType.repr(metatype) for metatype in metatypes ],
            columns=varnode_compare_level_labels
        )
        high_ratios_tablename = "metatype_match_levels_ratios"
        high_ratios_savepath = get_table_save_path(high_ratios_tablename, opts)
        high_ratios_df.to_csv(high_ratios_savepath)

        decomposed_raw_seriess = []
        decomposed_ratios_seriess = []
        for metatype in primitive_metatypes:
            metatype_str = MetaType.repr(metatype)
            grp = decomposed_varnodes_group_metatype(metatype)
            df = get_table_from_group(grp, opts)
            metatype_varnodes = df.iloc[:,0].sum()
            varnodes_by_levels = df.iloc[:,1:6].sum(axis=0)
            varnodes_by_levels.index = varnode_compare_level_labels
            level_ratios = varnodes_by_levels / metatype_varnodes
            decomposed_raw_seriess.append(varnodes_by_levels)
            decomposed_ratios_seriess.append(level_ratios)

        decomposed_raw_df = pd.DataFrame(
            decomposed_raw_seriess,
            index=[ MetaType.repr(metatype) for metatype in primitive_metatypes ],
            columns=varnode_compare_level_labels
        )
        decomposed_raw_tablename = "metatype_match_levels_decomposed"
        decomposed_raw_savepath = get_table_save_path(decomposed_raw_tablename, opts)
        decomposed_raw_df.to_csv(decomposed_raw_savepath)

        decomposed_ratios_df = pd.DataFrame(
            decomposed_ratios_seriess,
            index=[ MetaType.repr(metatype) for metatype in primitive_metatypes ],
            columns=varnode_compare_level_labels
        )
        decomposed_ratios_tablename = "metatype_match_levels_ratios_decomposed"
        decomposed_ratios_savepath = get_table_save_path(decomposed_ratios_tablename, opts)
        decomposed_ratios_df.to_csv(decomposed_ratios_savepath)

def get_metatype_match_levels_table(
    opts: BuildOptions,
    primitive: bool = False
) -> pd.DataFrame:
    tablename = "metatype_match_levels"
    if primitive:
        tablename += "_decomposed"
    return load_table_opts(tablename, opts)

def get_metatype_match_levels_ratios_table(
    opts: BuildOptions,
    primitive: bool = False
) -> pd.DataFrame:
    tablename = "metatype_match_levels_ratios"
    if primitive:
        tablename += "_decomposed"
    return load_table_opts(tablename, opts)

In [44]:
skip_generate_metatype_recovery_summaries = False

if not skip_generate_metatype_recovery_summaries:
    for opts in opts_sets:
        high_rows = {}
        for metatype in metatypes:
            row = {}
            df = get_table_from_group(varnodes_group_metatype(metatype), opts)
            # get the "ground truth" varnodes for metatype
            truth = df.iloc[:,0].sum()

            # get the varnode compare score for metatype
            level_sums = df.iloc[:,1:6].sum(axis=0)
            weights = np.array(list(VarnodeCompareLevel.range()))
            level_sums_weighted = np.multiply(weights, level_sums)
            level_sums_weighted_sum = level_sums_weighted.sum()
            score = level_sums_weighted_sum / (VarnodeCompareLevel.MATCH * truth)
            row["Varnode comparison score [0,1]"] = score

            missed = df.iloc[:,1].sum()
            matched = df.iloc[:,5].sum()
            row["Varnodes fraction partially recovered"] = (truth - missed) / truth
            row["Varnodes fraction exactly recovered"] = matched / truth
            high_rows[MetaType.repr(metatype)] = row

        high_df = pd.DataFrame.from_dict(high_rows, orient='index')
        high_tablename = "metatype_recovery_summary"
        high_savepath = get_table_save_path(high_tablename, opts)
        high_df.to_csv(high_savepath)

        decomposed_rows = {}
        for metatype in primitive_metatypes:
            row = {}
            df = get_table_from_group(decomposed_varnodes_group_metatype(metatype), opts)
            # get the "ground truth" varnodes for metatype
            truth = df.iloc[:,0].sum()

            # get the varnode compare score for metatype
            level_sums = df.iloc[:,1:6].sum(axis=0)
            weights = np.array(list(VarnodeCompareLevel.range()))
            level_sums_weighted = np.multiply(weights, level_sums)
            level_sums_weighted_sum = level_sums_weighted.sum()
            score = level_sums_weighted_sum / (VarnodeCompareLevel.MATCH * truth)
            row["Varnode comparison score [0,1]"] = score

            missed = df.iloc[:,1].sum()
            matched = df.iloc[:,5].sum()
            row["Varnodes fraction partially recovered"] = (truth - missed) / truth
            row["Varnodes fraction exactly recovered"] = matched / truth
            decomposed_rows[MetaType.repr(metatype)] = row

        decomposed_df = pd.DataFrame.from_dict(decomposed_rows, orient='index')
        decomposed_tablename = "metatype_recovery_summary_decomposed"
        decomposed_savepath = get_table_save_path(decomposed_tablename, opts)
        decomposed_df.to_csv(decomposed_savepath)

def get_metatype_recovery_summary_table(
    opts: BuildOptions,
    primitive: bool = False
) -> pd.DataFrame:
    tablename = "metatype_recovery_summary"
    if primitive:
        tablename += "_decomposed"
    return load_table_opts(tablename, opts)


In [45]:
skip_generate_metatype_recovery_with_levels_summaries = False

if not skip_generate_metatype_recovery_with_levels_summaries:
    for opts in opts_sets:
        for primitive in (True, False):
            dfs = [
                get_metatype_match_levels_table(opts, primitive=primitive),
                get_metatype_recovery_summary_table(opts, primitive=primitive)
            ]
            df = pd.concat(
                dfs,
                axis=1
            )
            tablename = "metatype_recovery_summary_with_levels"
            if primitive:
                tablename += "_decomposed"
            savepath = get_table_save_path(tablename, opts)
            df.to_csv(savepath)
                

def get_metatype_recovery_summary_with_levels_table(
    opts: BuildOptions,
    primitive: bool = False
) -> pd.DataFrame:
    tablename = "metatype_recovery_summary_with_levels"
    if primitive:
        tablename += "_decomposed"
    return load_table_opts(tablename, opts)

In [46]:
skip_generate_opts_varnodes_summary = False

if not skip_generate_opts_varnodes_summary:
    for primitive in (True, False):
        grp = decomposed_varnodes_group if primitive else varnodes_group
        _suffix = " (decomposed)" if primitive else ""
        seriess = []
        for opts in opts_sets:
            df = get_table_from_group(grp, opts)
            truth = df.iloc[:,0].sum()
            row = df.iloc[:,1:6].sum(axis=0)
            level_sums = row
            weights = np.array(list(VarnodeCompareLevel.range()))
            level_sums_weighted = np.multiply(weights, level_sums)
            level_sums_weighted_sum = level_sums_weighted.sum()
            score = level_sums_weighted_sum / (VarnodeCompareLevel.MATCH * truth)
            row["Varnode comparison score [0,1]" + _suffix] = score
            missed = df.iloc[:,1].sum()
            matched = df.iloc[:,5].sum()
            row["Varnodes fraction partially recovered" + _suffix] = (truth - missed) / truth
            row["Varnodes fraction exactly recovered" + _suffix] = matched / truth
            seriess.append(row)
        
        tablename = "opts_varnodes_summary"
        if primitive:
            tablename += "_decomposed"
        savepath = get_table_save_path_generic(tablename)
        
        df = pd.DataFrame(
            seriess,
            index=opts_sets_keys
        )
        for colname in df.columns[0:5]:
            df[colname] = df[colname].astype(int)
        df.to_csv(savepath)

def get_opts_varnodes_summary_table(primitive: bool = False) -> pd.DataFrame:
    tablename = "opts_varnodes_summary"
    if primitive:
        tablename += "_decomposed"
    df = load_table(tablename)
    return df

In [47]:
skip_generate_opts_varnodes_summary_metatypes = False

if not skip_generate_opts_varnodes_summary_metatypes:
    for primitive in (True, False):
        dfs = [ get_metatype_recovery_summary_with_levels_table(opts, primitive=primitive) for opts in opts_sets ]
        df = pd.concat(dfs, keys=opts_sets_keys, axis=0)
        tablename = "opts_varnodes_summary_metatypes"
        if primitive:
            tablename += "_decomposed"
        savepath = get_table_save_path_generic(tablename)
        df.to_csv(savepath)

def get_opts_varnodes_summary_metatypes_table(primitive: bool = False) -> pd.DataFrame:
    tablename = "opts_varnodes_summary_metatypes"
    if primitive:
        tablename += "_decomposed"
    savepath = get_table_save_path_generic(tablename)
    return pd.read_csv(savepath, index_col=[0,1])

In [48]:
skip_generate_opts_functions_summary = False

if not skip_generate_opts_varnodes_summary:
    rows = []
    for opts in opts_sets:
        df = get_table_from_group(functions_group, opts)
        row = df.iloc[:,0:3].sum(axis=0)
        rows.append(row)

    df = pd.DataFrame(rows, index=opts_sets_keys)
    df["Functions recovery fraction"] = df["Functions found"] / df["Ground truth functions"]

    savepath = get_table_save_path_generic("opts_functions_summary")
    df.to_csv(savepath)

def get_opts_functions_summary_table() -> pd.DataFrame:
    return load_table("opts_functions_summary")

In [49]:
skip_generate_opts_bytes_summary = False

if not skip_generate_opts_bytes_summary:
    rows = []
    for opts in opts_sets:
        df = get_table_from_group(bytes_group, opts)
        row = df.iloc[:,0:3].sum(axis=0)
        rows.append(row)

    df = pd.DataFrame(rows, index=opts_sets_keys)
    df["Bytes recovery fraction"] = df["Bytes found"] / df["Ground truth data bytes"]

    savepath = get_table_save_path_generic("opts_bytes_summary")
    df.to_csv(savepath)

def get_opts_bytes_summary_table() -> pd.DataFrame:
    return load_table("opts_bytes_summary")

In [50]:
df = get_table_from_group(varnodes_group_metatype(MetaType.UNION), debug_opts)
df

,Ground truth varnodes (metatype=UNION),Decompiler varnodes matched @ level=NO_MATCH (metatype=UNION),Decompiler varnodes matched @ level=OVERLAP (metatype=UNION),Decompiler varnodes matched @ level=SUBSET (metatype=UNION),Decompiler varnodes matched @ level=ALIGNED (metatype=UNION),Decompiler varnodes matched @ level=MATCH (metatype=UNION),"Varnode average compare score [0,1] (metatype=UNION)",Varnodes fraction partially recovered,Varnodes fraction exactly recovered
[,0,0,0,0,0,0,NaN,NaN,NaN
b2sum,0,0,0,0,0,0,NaN,NaN,NaN
base32,0,0,0,0,0,0,NaN,NaN,NaN
base64,0,0,0,0,0,0,NaN,NaN,NaN
basename,0,0,0,0,0,0,NaN,NaN,NaN
basenc,0,0,0,0,0,0,NaN,NaN,NaN
cat,0,0,0,0,0,0,NaN,NaN,NaN
chcon,0,0,0,0,0,0,NaN,NaN,NaN
chgrp,0,0,0,0,0,0,NaN,NaN,NaN
chmod,0,0,0,0,0,0,NaN,NaN,NaN


In [51]:
skip_generate_opts_array_comparisons_summary = False

if not skip_generate_opts_array_comparisons_summary:
    rows = []
    for opts in opts_sets:
        df = get_table_from_group(array_comparisons_group, opts)
        comparisons_col = df["Array comparisons"]
        total_comparisons = comparisons_col.sum()
        series0 = df.iloc[:,0:3].sum(axis=0)
        series1 = df.iloc[:,3:].transform(lambda col: col * comparisons_col, axis=0).sum(axis=0) / total_comparisons
        row = pd.concat((series0, series1), axis=0)
        rows.append(row)
    
    df = pd.DataFrame(
        rows,
        index=opts_sets_keys
    )
    df["Array varnodes inferred as array fraction"] = df["Array varnodes inferred as array"] / df["Ground truth varnodes (metatype=ARRAY)"]
    for colname in df.columns[0:3]:
        df[colname] = df[colname].astype(int)
    
    tablename = "opts_array_comparisons_summary"
    savepath = get_table_save_path_generic(tablename)
    df.to_csv(savepath)

def get_opts_array_comparisons_summary_table() -> pd.DataFrame:
    return load_table("opts_array_comparisons_summary")


In [52]:
skip_generate_opts_overall_summary = False

if not skip_generate_opts_overall_summary:
    df_functions = get_opts_functions_summary_table()
    df_varnodes = get_opts_varnodes_summary_table(primitive=False)
    df_primitive_varnodes = get_opts_varnodes_summary_table(primitive=True)
    df_bytes = get_opts_bytes_summary_table()

    functions_recovery_fraction = df_functions["Functions recovery fraction"]
    varnodes = df_varnodes.iloc[:,5:]
    varnodes_decomposed = df_primitive_varnodes.iloc[:,5:]
    bytes_recovery_fraction = df_bytes["Bytes recovery fraction"]

    df = pd.concat(
        (functions_recovery_fraction, varnodes, varnodes_decomposed, bytes_recovery_fraction),
        axis=1
    )

    tablename = "opts_overall_summary"
    savepath = get_table_save_path_generic(tablename)
    df.to_csv(savepath)

def get_opts_overall_summary_table() -> pd.DataFrame:
    return load_table("opts_overall_summary")

In [53]:
cmp = load_cmp(prog_from_progname("factor"), debug_opts)
union_cmp_records = [ 
    record for record in select_comparable_varnode_compare_records(cmp)
    if record.get_varnode().get_datatype().get_metatype() == MetaType.UNION
]
union = union_cmp_records[0].get_varnode().get_datatype()
for memtype in union.membertypes:
    print(memtype)

unsigned int4
<ARRAY subtype=char dimensions=(4096,) size=4096>


In [54]:
# Functions

display(get_opts_functions_summary_table())

,Ground truth functions,Functions found,Functions missed,Functions recovery fraction
strip,18139,18139,0,1.000000
standard,18139,18139,0,1.000000
debug,18139,18135,4,0.999779


In [55]:
# Varnodes

display(get_opts_varnodes_summary_table())
display(get_opts_varnodes_summary_metatypes_table())

,Varnodes matched @ level=NO_MATCH,Varnodes matched @ level=OVERLAP,Varnodes matched @ level=SUBSET,Varnodes matched @ level=ALIGNED,Varnodes matched @ level=MATCH,"Varnode comparison score [0,1]",Varnodes fraction partially recovered,Varnodes fraction exactly recovered
strip,1102,1662,1001,18612,12676,0.785981,0.968562,0.361624
standard,351,1450,613,19071,13568,0.814203,0.989987,0.387071
debug,125,52,24,43,34809,0.994672,0.996434,0.993039


Varnodes matched @ level=NO_MATCH  \
strip    INT                                     94   
         FLOAT                                    0   
         POINTER                                 68   
         ARRAY                                  788   
         STRUCT                                 152   
         UNION                                    0   
standard INT                                     51   
         FLOAT                                    0   
         POINTER                                 59   
         ARRAY                                  240   
         STRUCT                                   1   
         UNION                                    0   
debug    INT                                     41   
         FLOAT                                    0   
         POINTER                                 18   
         ARRAY                                   64   
         STRUCT                                   2   
         UNION                                    0   

                  Varnodes matched @ level=OVERLAP  \
strip    INT                                    48   
         FLOAT                                  56   
         POINTER                                 4   
         ARRAY                                 597   
         STRUCT                                955   
         UNION                                   2   
standard INT                                    48   
         FLOAT                                  56   
         POINTER                                 4   
         ARRAY                                 578   
         STRUCT                                762   
         UNION                                   2   
debug    INT                                    27   
         FLOAT                                   0   
         POINTER                                 0   
         ARRAY                                  17   
         STRUCT                                  8   
         UNION                                   0   

                  Varnodes matched @ level=SUBSET  \
strip    INT                                    0   
         FLOAT                                  0   
         POINTER                                0   
         ARRAY                                565   
         STRUCT                               432   
         UNION                                  4   
standard INT                                    0   
         FLOAT                                  0   
         POINTER                                0   
         ARRAY                                352   
         STRUCT                               257   
         UNION                                  4   
debug    INT                                    0   
         FLOAT                                  0   
         POINTER                                0   
         ARRAY                                 24   
         STRUCT                                 0   
         UNION                                  0   

                  Varnodes matched @ level=ALIGNED  \
strip    INT                                 12240   
         FLOAT                                 113   
         POINTER                              5840   
         ARRAY                                  19   
         STRUCT                                390   
         UNION                                  10   
standard INT                                 12284   
         FLOAT                                 113   
         POINTER                              5842   
         ARRAY                                  45   
         STRUCT                                777   
         UNION                                  10   
debug    INT                                    40   
         FLOAT                                   0   
         POINTER                                 1   
         ARRAY                                   0   
         STRUCT    

In [56]:
# Decomposed Varnodes

display(get_opts_varnodes_summary_table(primitive=True))
display(get_opts_varnodes_summary_metatypes_table(primitive=True))

,Varnodes matched @ level=NO_MATCH (decomposed),Varnodes matched @ level=OVERLAP (decomposed),Varnodes matched @ level=SUBSET (decomposed),Varnodes matched @ level=ALIGNED (decomposed),Varnodes matched @ level=MATCH (decomposed),"Varnode comparison score [0,1] (decomposed)",Varnodes fraction partially recovered (decomposed),Varnodes fraction exactly recovered (decomposed)
strip,139937,31280,0,231309,131719,0.585912,0.738066,0.246552
standard,40348,56605,0,303569,133723,0.702957,0.924477,0.250303
debug,10708,128,0,41,523368,0.979758,0.979957,0.979640


Varnodes matched @ level=NO_MATCH  \
strip    INT                                 133056   
         FLOAT                                   72   
         POINTER                               6740   
standard INT                                  40163   
         FLOAT                                    0   
         POINTER                                147   
debug    INT                                  10679   
         FLOAT                                    0   
         POINTER                                 29   

                  Varnodes matched @ level=OVERLAP  \
strip    INT                                 28812   
         FLOAT                                  73   
         POINTER                              2057   
standard INT                                 46846   
         FLOAT                                 145   
         POINTER                              9245   
debug    INT                                   124   
         FLOAT                                   0   
         POINTER                                 2   

                  Varnodes matched @ level=SUBSET  \
strip    INT                                    0   
         FLOAT                                  0   
         POINTER                                0   
standard INT                                    0   
         FLOAT                                  0   
         POINTER                                0   
debug    INT                                    0   
         FLOAT                                  0   
         POINTER                                0   

                  Varnodes matched @ level=ALIGNED  \
strip    INT                                217959   
         FLOAT                                 103   
         POINTER                             13214   
standard INT                                290472   
         FLOAT                                 103   
         POINTER                             12961   
debug    INT                                    40   
         FLOAT                                   0   
         POINTER                                 1   

                  Varnodes matched @ level=MATCH  \
strip    INT                              125208   
         FLOAT                                22   
         POINTER                            6409   
standard INT                              127554   
         FLOAT                                22   
         POINTER                            6067   
debug    INT                              494192   
         FLOAT                               270   
         POINTER                           28388   

                  Varnode comparison score [0,1]  \
strip    INT                            0.585861   
         FLOAT                          0.435185   
         POINTER                        0.592321   
standard INT                            0.707118   
         FLOAT                          0.501852   
         POINTER                        0.636840   
debug    INT                            0.978651   
         FLOAT                          1.000000   
         POINTER                        0.998918   

                  Varnodes fraction partially recovered  \
strip    INT                                   0.736541   
         FLOAT                                 0.733333   
         POINTER                               0.762843   
standard INT                                   0.920475   
         FLOAT                                 1.000000   
         POINTER                               0.994828   
debug    INT                                   0.978855   
         FLOAT                                 1.000000   
         POINTER                               0.998980   

                  Varnodes fraction exactly recovered  
strip    INT                                 0.247919  
         FLOAT                               0.081481  
         POINTER                             0.225510

In [57]:
# Data Bytes

display(get_opts_bytes_summary_table())

,Ground truth data bytes,Bytes found,Bytes missed,Bytes recovery fraction
strip,1185783,955053,230730,0.805420
standard,1185783,955053,230730,0.805420
debug,1185783,1178169,7614,0.993579


In [58]:
# Summary

display(get_opts_overall_summary_table())

,Functions recovery fraction,"Varnode comparison score [0,1]",Varnodes fraction partially recovered,Varnodes fraction exactly recovered,"Varnode comparison score [0,1] (decomposed)",Varnodes fraction partially recovered (decomposed),Varnodes fraction exactly recovered (decomposed),Bytes recovery fraction
strip,1.000000,0.785981,0.968562,0.361624,0.585912,0.738066,0.246552,0.805420
standard,1.000000,0.814203,0.989987,0.387071,0.702957,0.924477,0.250303,0.805420
debug,0.999779,0.994672,0.996434,0.993039,0.979758,0.979957,0.979640,0.993579


In [59]:
# Array Comparisons

display(get_opts_array_comparisons_summary_table())

,Ground truth varnodes (metatype=ARRAY),Array comparisons,Array varnodes inferred as array,Array varnodes inferred as array fraction,Array length (elements) average error,Array length (elements) average error ratio,Array size (bytes) average error,Array size (bytes) average error ratio,"Array dimension match score [0,1]","Array average element type comparison score [0,1]"
strip,2197,823,774,0.352299,134.695018,2.844632,458.574727,0.911925,0.979344,0.781288
standard,2197,1579,1530,0.696404,151.155795,5.441502,239.022799,0.474756,0.974668,0.669833
debug,2197,2226,2128,0.968594,9.415993,0.109666,9.415993,0.109666,1.000000,0.999102


In [60]:
# cmp = load_cmp(prog_from_progname("cksum"), debug_opts)
# truth = sum([ varnode.get_size() for varnode in varnodes_truth(cmp) ])
# missed = sum([ varnode.get_size() for varnode in varnodes_missed(cmp) ])
# overlapped = varnode_compare_records_matched_at_level(cmp, VarnodeCompareLevel.OVERLAP)
# for varnode in varnodes_missed(cmp):
#     print(varnode.get_var().get_parent_function().get_name())

# for record in overlapped:
#     print(record.get_varnode().get_var().get_parent_function().get_name())
# print(overlapped)